이미지에 관한 계산을 하기 위한 컨볼루션 신경망 cnn  
stride = 보폭       필터를 만들 때   
  
통째 학습  
특징학습  
신셩망 깊이를 깊게 하여 풍부한 특징을 추출: 수십 ~ 수백 층 사용  
데이터의 원래 구조를 유지한 채 특징 추출  

Convolution Layer  (함성곱 층)
합성 곱을 해서 이미지에서 중요한 부분을 뽑아내는 것.    이미지 * 필터(매트릭스)

    stride(보폭)    
    필터(매트릭스)를 얼마만큼 움직일 것인가를 지정하는 값으로
    필터링을 하고 나온 output은 stride에 따라서 값이 다르게 나오게 된다. 
    필터를 적용하는 위치의 간격 

    padding(패딩, 덧대기)
    합성곱을 하게 되면 이미지 크기가 줄어들게 된다. 이것을 해결하기 위해 매트릭스의 가장자리 모든 픽셀에다 쓸모없는 값을 덧대서 가장자리에도 
    연산을 하고, 이미지 크기가 줄어드는 것도 방지를 한다. 


Pooling Layer   (풀링 층)
풀링 층의 목적은 과도한 계산량, 메모리 사용량, 파라미터 수를 줄이고 결과적으로 over fitting의 위험을 줄여주는 축소본을 만드는 것입니다.
즉, 최대나 평균 같은 계산만 하는 것이 풀링층입니다. 
    feature map(이미지)의 지나친 상세함을 줄여서 요약한 통계량을 추출해서 사용을 하게 된다. 
    즉, 이미지(feature map)에 윈도우를 지정을 하고 그 윈도우 안에서 가장 큰 값을 가져오는 것으로 공간 크기를 줄여줍니다. 
        윈도우 지정은 stride로 지정 할 수 있습니다. 
            stride가 2면 윈도우 2 * 2 
            stride가 3이면 3 * 3의 형태로 output feature map(아웃풋 이미지)가 나오게 된다.    

풀링층: 최대 풀링은 커널 안에 있는 화소 중에 최대값을 취함(평균 풀링은 평균을 취함)
특징 맵에 있는 지나친 상세함을 줄여 요약 통계량을 추출해 줌 
보폭(stride)를 n으로 하면 이미지맵은 n만큼 줄어드는 모습을 보여줍니다. 

In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
print(keras.__version__)

C:\Users\vlrms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)


2.10.0
2.10.0


In [1]:
# 풀링층 이미지에 필터를 적용하여 이미지를 줄이고, 대표되는 값을 가져오는 것 / 보폭 = stride 

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

# MNIST 데이터셋을 읽고 신경망에 입력할 형태로 변환
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

###############################################
# 신경망 모델 설계
cnn=Sequential()  # <----- cnn 선언 
                
                #3*3 필터를 32번 사용한다. activation=‘relu’는 컨볼루션 결과에 ReLU 활성 함수를 적용하라는 뜻
                # input_shape=(28,28,1) 매개변수는 신경망에 (28*28*1) 텐서가 입력된다는 사실 알려줌 
                # (28*28) 대신 (28*28*1)을 사용하는 이유는 일반성 유지(RGB 영상의 경우 (28*28*3)으로 확장)  
cnn.add(Conv2D(32,(3,3),activation='relu', input_shape=(28,28,1)))
cnn.add(Conv2D(64,(3,3), activation='relu'))

# 블록 내의 원소들 중 최대값을 대표값으로 선택하는 Max Pooling
cnn.add(MaxPooling2D(pool_size=(2,2)))  # 필터링 

#Dropout    일정 비율의 가중치를 임의로 선택하여 불능으로 만들고 학습하는 규제기법
# 불능이 될 샘플마다 독립적으로 정하는데 난수를 통해 랜덤하게 선택함. 
# 즉, 랜덤한 노드를 빼버림 
cnn.add(Dropout(0.25))

#컴퓨터가 인식할 수 있게 1차원으로 만듦
cnn.add(Flatten())

#완전연결층
cnn.add(Dense(128, activation='relu'))

#Dropout
cnn.add(Dropout(0.5))

#완전연결층
cnn.add(Dense(10,activation='softmax'))


#신경망 모델 학습
cnn.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
hist = cnn.fit(x_train,y_train,batch_size=128, epochs=12, validation_data=(x_test,y_test), verbose=2)


# 신경망 모델 정확률 평가


res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

import matplotlib.pyplot as plt



# 정확률 그래프
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'], loc='best')
plt.grid()
plt.show()

# 손실 함수 그래프
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Modelloss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'],loc='best')
plt.grid()
plt.show()


#학습된 모델 저장
#   cnn.save("파일이름")    구조정보와 가중치를 저장해준다. 
# 불러오기
# cnn=tf.keras.model/load_model("파일명.h5")

C:\Users\vlrms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)


Epoch 1/12
469/469 - 61s - loss: 0.2419 - accuracy: 0.9264 - val_loss: 0.0514 - val_accuracy: 0.9828 - 61s/epoch - 130ms/step
Epoch 2/12
469/469 - 86s - loss: 0.0869 - accuracy: 0.9739 - val_loss: 0.0369 - val_accuracy: 0.9880 - 86s/epoch - 184ms/step
Epoch 3/12
469/469 - 84s - loss: 0.0649 - accuracy: 0.9801 - val_loss: 0.0352 - val_accuracy: 0.9886 - 84s/epoch - 178ms/step
Epoch 4/12
469/469 - 103s - loss: 0.0544 - accuracy: 0.9831 - val_loss: 0.0346 - val_accuracy: 0.9893 - 103s/epoch - 219ms/step
Epoch 5/12
469/469 - 94s - loss: 0.0454 - accuracy: 0.9852 - val_loss: 0.0283 - val_accuracy: 0.9906 - 94s/epoch - 200ms/step
Epoch 6/12
469/469 - 95s - loss: 0.0398 - accuracy: 0.9876 - val_loss: 0.0283 - val_accuracy: 0.9916 - 95s/epoch - 202ms/step
Epoch 7/12
469/469 - 91s - loss: 0.0365 - accuracy: 0.9884 - val_loss: 0.0303 - val_accuracy: 0.9905 - 91s/epoch - 194ms/step
Epoch 8/12


KeyboardInterrupt: 

In [ ]:
# cnn의 합성곱에 관한 함수 https://amber-chaeeunk.tistory.com/31
# 책도 같이 보기 

import numpy as np

def im2col(nput_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant', constant_values=0)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

x1 = np.random.rand(1, 3, 7, 7) #데이터 수, 채널 수, 높이, 너비
col1 = im2col(x1, 5, 5, stride = 1, pad = 0) #데이터 수, 필터 높이, 필터 너비, 스트라이드, 패딩)
print(col1.shape) #(9, 75) 

x2 = np.random.rand(10, 3, 7, 7) #데이터 10개
col2 = im2col(x2, 5, 5, strid = 1, pad = 0)
print(col2.shape) #(90, 75)